<a href="https://colab.research.google.com/github/jaadu-1/Algo-trading/blob/main/decision_track_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

krish39696_decision_track_path = kagglehub.dataset_download('krish39696/decision-track')
krish39696_krish_data_path = kagglehub.dataset_download('krish39696/krish-data')

print('Data source import complete.')


In [ ]:
!pip install lightgbm

import pandas as pd
import pyarrow.parquet as pq
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
# Base data path
DATA_PATH = "/kaggle/input/krish-data"


In [ ]:
# Core datasets
train = pd.read_parquet(f"{DATA_PATH}/train_data.parquet")
test = pd.read_parquet(f"{DATA_PATH}/test_data.parquet")
submission = pd.read_csv(f"{DATA_PATH}/685404e30cfdb_submission_template.csv")

# Supplementary files
events = pd.read_parquet(f"{DATA_PATH}/add_event.parquet")
offers = pd.read_parquet(f"{DATA_PATH}/offer_metadata.parquet")
data_dict = pd.read_csv(f"{DATA_PATH}/data_dictionary.csv")

# This file was uploaded separately
transactions = pd.read_parquet("/kaggle/input/krish-data/add_trans.parquet")

print("✅ All 7 datasets loaded successfully.")


✅ All 7 datasets loaded successfully.


In [ ]:
print("Train:", train.shape)
print("Test:", test.shape)
print("Events:", events.shape)
print("Offers:", offers.shape)
print("Transactions:", transactions.shape)
print("Submission template:", submission.shape)
print("Data Dictionary:", data_dict.shape)


Train: (770164, 372)
Test: (369301, 371)
Events: (21457473, 5)
Offers: (4164, 12)
Transactions: (6339465, 9)
Submission template: (369301, 5)
Data Dictionary: (372, 3)


In [ ]:
print("\nEvents Columns:\n", events.columns.tolist())
print("\nTransactions Columns:\n", transactions.columns.tolist())
print("\nOffers Columns:\n", offers.columns.tolist())
print("\nSubmission Columns:\n", submission.columns.tolist())



Events Columns:
 ['id2', 'id3', 'id6', 'id4', 'id7']

Transactions Columns:
 ['id2', 'f367', 'f368', 'f369', 'f370', 'f371', 'f372', 'id8', 'f374']

Offers Columns:
 ['id3', 'id9', 'f375', 'f376', 'f377', 'id10', 'id11', 'f378', 'f374', 'id8', 'id12', 'id13']

Submission Columns:
 ['id1', 'id2', 'id3', 'id5', 'pred']


In [ ]:
print("\nSample from Events:\n", events.head())
print("\nSample from Transactions:\n", transactions.head())
print("\nSample from Offers:\n", offers.head())



Sample from Events:
        id2     id3    id6                      id4   id7
0  2431360  618619  Tiles  2023-10-22 08:08:17.768  None
1  2431360  363153  Tiles  2023-10-22 08:08:18.921  None
2  2431360   97193  Tiles  2023-10-22 08:08:17.765  None
3  2431360  654444  Tiles  2023-10-22 08:08:17.737  None
4  2431360   32325  Tiles  2023-10-22 08:08:17.812  None

Sample from Transactions:
        id2   f367 f368 f369        f370      f371    f372       id8 f374
0  2896709  15.60  PBR    D  2023-10-16  19:16:52  202310  59639998  DSE
1  2855047   6.40   PR    D  2023-10-14  13:01:16  202310  59639998  DSE
2  2497175  13.99  PBR    D  2023-10-14  00:31:48  202310  59639998  DSE
3  2655364  15.14  PGC    D  2023-10-13  12:37:25  202310  59639998  DSE
4  2855047   2.12   PR    D  2023-10-09  16:51:21  202310  59639998  DSE

Sample from Offers:
          id3   id9  f375   f376  f377 id10  id11 f378  f374   id8  \
0      70687  FO5O     2    5.0  None    1  None    N  None  None   
1  9000025

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [ ]:
print("Train Columns:\n", train.columns.tolist())
print("\nTest Columns:\n", test.columns.tolist())

Train Columns:
 ['id1', 'id2', 'id3', 'id4', 'id5', 'y', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f1

In [ ]:
print("\nSample from Train:\n", train.head())


Sample from Train:
                                                id1      id2        id3  \
0  1366776_189706075_16-23_2023-11-02 22:22:00.042  1366776  189706075   
1      1366776_89227_16-23_2023-11-01 23:51:24.999  1366776      89227   
2      1366776_35046_16-23_2023-11-01 00:30:59.797  1366776      35046   
3    1366776_6275451_16-23_2023-11-02 22:21:32.261  1366776    6275451   
4      1366776_78053_16-23_2023-11-02 22:21:34.799  1366776      78053   

                       id4         id5  y   f1    f2    f3    f4  ...  f357  \
0  2023-11-02 22:22:00.042  2023-11-02  0  1.0  None  None  None  ...  None   
1  2023-11-01 23:51:24.999  2023-11-01  0  1.0  None  None  None  ...  None   
2  2023-11-01 00:30:59.797  2023-11-01  0  1.0  None  None  None  ...  None   
3  2023-11-02 22:21:32.261  2023-11-02  0  1.0  None  None  None  ...  None   
4  2023-11-02 22:21:34.799  2023-11-02  0  1.0  None  None  None  ...  None   

      f358 f359  f360   f361 f362               f363    f36

In [ ]:
# ====================
# STEP X: Preprocessing
# ====================

from sklearn.preprocessing import LabelEncoder

# Separate column types
num_cols = train.select_dtypes(include=['float64', 'int64']).columns
cat_cols = train.select_dtypes(include=['object']).columns

# Ensure we drop the target variable if it's in cat_cols
if 'y' in cat_cols:
    cat_cols = cat_cols.drop('y')

# Handle numeric columns (fill missing with mean)
for col in num_cols:
    if col != 'y':  # don't fill target column
        train[col] = train[col].fillna(train[col].mean())
        test[col] = test[col].fillna(train[col].mean())

# Handle categorical columns (label encoding)
for col in cat_cols:
    if col in test.columns:
        le = LabelEncoder()
        combined = pd.concat([train[col], test[col]], axis=0).astype(str)
        le.fit(combined)
        train[col] = le.transform(train[col].astype(str))
        test[col] = le.transform(test[col].astype(str))
    else:
        print(f"⚠️ Skipping {col} - not found in test set.")

print("✅ Preprocessing done.")


✅ Preprocessing done.
